# Entrenamiento de clasificadores avanzados en *scikit-learn*
En este notebook aprenderás a entrenar clasificadores avanzados en Python gracias a la librería *scikit-learn*. Además, también se compararán en distintos escenarios a los clasificadores más convencionales, tanto en rendimiento predictivo como en complejidad. Así, se tratará de discernir qué método (o métodos) es mejor o más recomendable en cada caso.

Es recomendable tener en todo momento disponible la [Guía de usuario de *scikit-learn*](https://scikit-learn.org/stable/user_guide.html), o la documentación de la [API](https://scikit-learn.org/stable/modules/classes.html).

## 1.   Carga de datos

A lo largo del notebook, en este caso, vamos a utilizar 4 conjuntos de datos distintos que cargaremos en esta primer celda. A su vez, cuando sea necesario, realizaremos particiones de entrenamiento y test. Algunos conjuntos de datos se utilizan ya en las distintas celdas de código; para el resto de datasets se aconseja al estudiante que modifique el código necesario para poder realizar los mismos procesos con distintos conjuntos de datos.

En primer lugar, como en casos anteriores, el conjunto de datos de [*Breast Cancer*](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html). Este conjunto de datos de juguete contiene 569 patrones en total, cada uno descrito por 30 atributos, y con dos clases de salida.

En segundo lugar, vamos a cargar un conjunto de datos de otro problema binario, pero siendo en este caso un problema real: el dataset [Labeled Faces in the Wild (pairs)](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_lfw_pairs.html#sklearn.datasets.fetch_lfw_pairs). Este conjunto de datos contiene información de imágenes de caras de personas famosas. Uno de los problemas para este conjunto de datos sería [identificar el nombre de la persona dada la imagen](https://scikit-learn.org/stable/datasets/real_world.html#labeled-faces-in-the-wild-dataset). Sin embargo, en el conjunto [*pairs*](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_lfw_pairs.html#sklearn.datasets.fetch_lfw_pairs), que es el que utilizaremos en este notebook, cada instancia contiene información de dos caras, y la clase es binaria, indicando si se trata o no de la misma persona. El conjunto de datos incluye más de 13000 patrones descritos cada uno con más de 5800 atributos de entrada (correspondientes a dos imágenes de 62*47 pixels).

También utilizaremos otro problema real de clasificación multi-clase. El conjunto de datos [Olivetti Faces](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_olivetti_faces.html#sklearn.datasets.fetch_olivetti_faces) contiene también imágenes de 40 personas en distintos escenarios de luz, expresión facial, etc. La tarea se basa en identificar la identidad de la persona dada su imagen. Este conjunto de datos contiene 400 patrones descritos por 4096 atributos de entrada (imágenes de 92*112 pixels), y un total de 40 clases distintas.

Por último, utilizaremos el dataset [MNIST](https://www.openml.org/d/554) para reconocimiento de dígitos manuscritos. Este conjunto se describió en la tercera lección del curso, pero puede encontrar más información [aquí](http://yann.lecun.com/exdb/mnist/). El conjunto de datos contiene 60000 patrones de entrenamiento y 10000 de test, cada uno de ellos descrito por 784 atributos de entrada (imágenes de 28*28 pixels), y un total de 10 clases (dígitos del 0 al 9).


In [1]:
from sklearn.datasets import fetch_lfw_pairs, fetch_olivetti_faces, load_breast_cancer
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.utils import check_random_state
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

# Lo utilizaremos para escalar todas las variables al mismo rango
scaler = StandardScaler()

# Cargamos el conjunto de datos Cancer y dividimos en train-test
cancer_X, cancer_y = load_breast_cancer(return_X_y=True)
cancer_X_train, cancer_X_test, cancer_y_train, cancer_y_test = train_test_split(cancer_X, cancer_y, test_size=0.2, random_state=0)

cancer_X_train = scaler.fit_transform(cancer_X_train)
cancer_X_test = scaler.transform(cancer_X_test)

# Cargamos el conjunto de LFW, que se proporciona directamente en conjuntos de entranamiento y test
lfw_train = fetch_lfw_pairs(subset='train')
lfw_X_train = lfw_train.data
lfw_y_train = lfw_train.target
lfw_test = fetch_lfw_pairs(subset='test')
lfw_X_test = lfw_test.data
lfw_y_test = lfw_test.target

lfw_X_train = scaler.fit_transform(lfw_X_train)
lfw_X_test = scaler.transform(lfw_X_test)

# Cargamos el conjunto de olivetti faces y dividimos en train-test
olivetti_X, olivetti_y = fetch_olivetti_faces(return_X_y=True)
olivetti_X_train, olivetti_X_test, olivetti_y_train, olivetti_y_test = train_test_split(olivetti_X, olivetti_y, test_size=0.2, random_state=0)

olivetti_X_train = scaler.fit_transform(olivetti_X_train)
olivetti_X_test = scaler.transform(olivetti_X_test)

# Cargamos y preprocesamos dataset MNIST
mnist_X, mnist_y = fetch_openml("mnist_784", return_X_y=True, as_frame=False)
# Originalmente serian 60000 instancias de entrenamiento y 10000 de test, pero para hacer el proceso más rápido, dejaremos 6000, y 1000 de test
mnist_X_train, mnist_X_test, mnist_y_train, mnist_y_test = train_test_split(mnist_X, mnist_y, train_size=5000, test_size=1000, random_state=0)

mnist_X_train = scaler.fit_transform(mnist_X_train)
mnist_X_test = scaler.transform(mnist_X_test)


## 2.   Entrenamiento de modelos avanzados

### 2.1.  SVM

En primer lugar, vamos a entrenar modelos de SVM. Para ello, vamos a realizar un proceso de búsqueda de parámetros en dos fases: en primer lugar, entrenaremos el [SVM](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) con distintos kernels; y en segundo lugar, para el kernel que mejor funcionamiento tuviese, optimizaremos su valor de C.

En primer lugar, ejecutamos SVM con kernels lineal, polinómico (por defecto, grado 3), RBF (funciones gausianas), y sigmoide.

En segundo lugar, utilizamos valores de C desde 1e-3 hasta 1e3; mayores valores de C hacen preferible un margen menor siempre que los datos de entrenamiento se clasifiquen mejor; valores menores de C obtienen márgenes mayores pese a que la frontera sea más simple y suponga una pérdida en rendimiento predictivo.

In [2]:
import timeit

from sklearn import metrics
from sklearn import svm

# Buscar kernel con mejor rendimiento
kernels = ['linear', 'poly', 'rbf', 'sigmoid']
times = [-1] * len(kernels)
acc = [-1] * len(kernels)
f1 = [-1] * len(kernels)

for i in range(len(kernels)):
  print(kernels[i])
  tic = timeit.default_timer()
  svm_model = svm.SVC(kernel=kernels[i]).fit(lfw_X_train, lfw_y_train)
  toc = timeit.default_timer()
  times[i] = toc - tic

  svm_pred = svm_model.predict(lfw_X_test)

  acc[i] = metrics.accuracy_score(lfw_y_test, svm_pred)
  f1[i] = metrics.f1_score(lfw_y_test, svm_pred)

print('\nResultados')
print(kernels)
print(times)
print(acc)
print(f1)

linear
poly
rbf
sigmoid

Resultados
['linear', 'poly', 'rbf', 'sigmoid']
[7.696641249999999, 4.837471208000004, 4.754057249999988, 3.936020499999998]
[0.525, 0.563, 0.626, 0.483]
[0.5329400196656834, 0.6261762189905903, 0.6375968992248061, 0.4916420845624386]


In [3]:
import timeit

from sklearn import metrics
from sklearn import svm

# Buscar mejor valor de C para un kernel concreto
Cs = [1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3]

times = [-1] * len(Cs)
acc = [-1] * len(Cs)
f1 = [-1] * len(Cs)

for i in range(len(Cs)):
  print(Cs[i])
  tic = timeit.default_timer()
  svm_model = svm.SVC(kernel='rbf', C=Cs[i]).fit(lfw_X_train, lfw_y_train)
  toc = timeit.default_timer()
  times[i] = toc - tic

  svm_pred = svm_model.predict(lfw_X_test)

  acc[i] = metrics.accuracy_score(lfw_y_test, svm_pred)
  f1[i] = metrics.f1_score(lfw_y_test, svm_pred)

print('\nResultados')
print(Cs)
print(times)
print(acc)
print(f1)

0.001
0.01
0.1
1.0
10.0
100.0
1000.0

Resultados
[0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
[5.109994084000007, 5.385730707999997, 5.1610819590000006, 4.844310874999991, 5.31380320800001, 5.19716112499998, 5.6544541249999725]
[0.587, 0.587, 0.601, 0.626, 0.617, 0.617, 0.617]
[0.5693430656934307, 0.5693430656934307, 0.6159769008662176, 0.6375968992248061, 0.6277939747327502, 0.6277939747327502, 0.6277939747327502]


### 2.2. Random Forest

Ahora, vamos a entrenar distintos modelos de [Random Forest](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier). En este caso, vamos a variar el número de árboles del ensemble y la profundidad máxima de cada uno de ellos, siguiendo un método de rejilla o *grid* (es decir, probar todas las combinaciones de ambos parámetros, no como en el caso del SVM).


In [4]:
from sklearn.ensemble import RandomForestClassifier

# Número de árboles en el ensemble
n = [10, 50, 100, 500, 1000]

# Profundidad máxima de cada árbol
max_depth_trees = [None, 3, 5]

times = [None] * len(n)
acc = [None] * len(n)
f1 = [None] * len(n)


for i in range(len(n)):
  print(i)
  times[i] = [-1] * len(max_depth_trees)
  acc[i] = [-1] * len(max_depth_trees)
  f1[i] = [-1] * len(max_depth_trees)
  for j in range(len(max_depth_trees)):
    print('  ' + str(j))
    tic = timeit.default_timer()
    rf = RandomForestClassifier(n_estimators=n[i], max_depth=max_depth_trees[j]).fit(lfw_X_train, lfw_y_train)
    toc = timeit.default_timer()
    times[i][j] = toc - tic

    rf_pred = rf.predict(lfw_X_test)

    acc[i][j] = metrics.accuracy_score(lfw_y_test, rf_pred)
    f1[i][j] = metrics.f1_score(lfw_y_test, rf_pred)

print('\nResultados')
print(times)
print(acc)
print(f1)

0
  0
  1
  2
1
  0
  1
  2
2
  0
  1
  2
3
  0
  1
  2
4
  0
  1
  2

Resultados
[[0.7331963750000057, 0.19958862500001828, 0.3305500410000093], [3.699524083, 0.9892573329999834, 1.5993371669999874], [7.40552466699998, 1.9613925829999914, 3.145623124999986], [36.99846208300002, 9.83803341700002, 16.049145208], [73.363579958, 19.446321875000024, 31.734129333]]
[[0.577, 0.549, 0.551], [0.559, 0.56, 0.567], [0.602, 0.576, 0.59], [0.61, 0.591, 0.592], [0.607, 0.592, 0.589]]
[[0.5187713310580205, 0.5547877591312932, 0.5619512195121951], [0.5620655412115194, 0.5537525354969575, 0.568295114656032], [0.5930470347648262, 0.5867446393762183, 0.5964566929133858], [0.6161417322834645, 0.5930348258706467, 0.6023391812865496], [0.6158357771260997, 0.6015625, 0.6044273339749759]]


### 2.3. Redes neuronales 

Como último clasificador avanzado, entrenaremos un modelo de Red Neuronal Artificial, llamado [MLP - *Multi-Layer Perceptron*](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html) en *scikit-learn*).

En este caso, vamos a variar el número de nodos en capa oculta y el número de capas ocultas, entre algunos valores arbitrarios escogidos. Hay muchos otros parámetros que podían optimizarse, pero vamos a centrarnos en este.

In [5]:
from sklearn.neural_network import MLPClassifier

n_attr = 5828

# Distintas configuraciones de capas ocultas
# Cada tupla indica el número de neuronas en cada una de las capas ocultas
hidden = [(10,), (50,), (50, 10,), (100,), (100, 50,), (round(math.sqrt(n_attr)),), (round(math.sqrt(n_attr)), round(math.log2(n_attr)))]
times = [None] * len(hidden)
acc = [None] * len(hidden)
f1 = [None] * len(hidden)

for i in range(len(hidden)):
  print(hidden[i])
  tic = timeit.default_timer()
  mlp = MLPClassifier(hidden_layer_sizes=hidden[i], random_state=0).fit(lfw_X_train, lfw_y_train)
  toc = timeit.default_timer()

  times[i] = toc - tic

  mlp_pred = mlp.predict(lfw_X_test)

  acc[i] = metrics.accuracy_score(lfw_y_test, mlp_pred)
  f1[i] = metrics.f1_score(lfw_y_test, mlp_pred)

print('\nResultados')
print(hidden)
print(times)
print(acc)
print(f1)


NameError: name 'math' is not defined

## 3. Entrenamiento de modelos clásicos y comparación

Posteriormente, vamos a entrenar varios modelos clásicos, como regresión logística, kNN, árbol de decisión, y un Naive Bayes.

In [ ]:
import timeit

from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.naive_bayes import GaussianNB

# Entrenar clasificadores clásicos con conjunto de datos de lfw
tic = timeit.default_timer()
lr = LogisticRegression(random_state=0).fit(lfw_X_train, lfw_y_train)
toc = timeit.default_timer()
print('LR train time: ' + str(toc - tic))

lr_pred = lr.predict(lfw_X_test)

print('LR acc: ' + str(metrics.accuracy_score(lfw_y_test, lr_pred)))
print('LR F1: ' + str(metrics.f1_score(lfw_y_test, lr_pred)))
print('---')

tic = timeit.default_timer()
tree = tree.DecisionTreeClassifier().fit(lfw_X_train, lfw_y_train)
toc = timeit.default_timer()
print('Tree train time: ' + str(toc - tic))

tree_pred = tree.predict(lfw_X_test)

print('Tree acc: ' + str(metrics.accuracy_score(lfw_y_test, tree_pred)))
print('Tree F1: ' + str(metrics.f1_score(lfw_y_test, tree_pred)))
print('---')

tic = timeit.default_timer()
gnb = GaussianNB().fit(lfw_X_train, lfw_y_train)
toc = timeit.default_timer()
print('GNB train time: ' + str(toc - tic))

gnb_pred = gnb.predict(lfw_X_test)

print('GNB acc: ' + str(metrics.accuracy_score(lfw_y_test, gnb_pred)))
print('GNB F1: ' + str(metrics.f1_score(lfw_y_test, gnb_pred)))
print('---')

# Tener en cuenta que con ciertos conjuntos de datos puede ser MUY costoso obtener las predicciones en test
tic = timeit.default_timer()
knn = KNeighborsClassifier(n_neighbors=3).fit(lfw_X_train, lfw_y_train)
toc = timeit.default_timer()
print('kNN train time: ' + str(toc - tic))

tic = timeit.default_timer()
knn_pred = knn.predict(lfw_X_test)
toc = timeit.default_timer()
print('kNN testing time: ' + str(toc - tic))

print('kNN acc: ' + str(metrics.accuracy_score(lfw_y_test, knn_pred)))
print('kNN F1: ' + str(metrics.f1_score(lfw_y_test, knn_pred)))
print('---')


Compare los resultados de los distintos métodos (en los métodos avanzados, tenga en cuenta el resultado del mejor modelo obtenido).


*   ¿Qué métodos obtienen los mejores resultados? ¿Los avanzados o los clásicos?
*   En caso de que un método avanzado sea mejor que los clásicos, compare los tiempos de entrenamiento. Si el tiempo del método avanzado es mayor que el del clásico, ¿merece la pena ese incremento en tiempo para el incremento en rendimiento predictivo?
*   Fijese en el rendimiento predictivo del árbol de decisión y de Random Forest (ensemble de árboles de decisión). ¿Mejora Random Forest los resultados del árbol de decisión? Es decir, consigue un método de ensemble superar el rendimiento de su clasificador base?



## 4. Análisis en otro conjunto de datos

Vamos a realizar ahora el mismo proceso pero con un conjunto de datos distinto.

In [ ]:
import timeit

from sklearn import metrics
from sklearn import svm

# Buscar kernel con mejor rendimiento
kernels = ['linear', 'poly', 'rbf', 'sigmoid']
times = [-1] * len(kernels)
acc = [-1] * len(kernels)
f1 = [-1] * len(kernels)

for i in range(len(kernels)):
  print(kernels[i])
  tic = timeit.default_timer()
  svm_model = svm.SVC(kernel=kernels[i]).fit(mnist_X_train, mnist_y_train)
  toc = timeit.default_timer()
  times[i] = toc - tic

  svm_pred = svm_model.predict(mnist_X_test)

  acc[i] = metrics.accuracy_score(mnist_y_test, svm_pred)
  f1[i] = metrics.f1_score(mnist_y_test, svm_pred, average='macro')

print('\nResultados')
print(kernels)
print(times)
print(acc)
print(f1)

In [ ]:
import timeit

from sklearn import metrics
from sklearn import svm

# Buscar mejor valor de C para un kernel concreto
Cs = [1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3]

times = [-1] * len(Cs)
acc = [-1] * len(Cs)
f1 = [-1] * len(Cs)

for i in range(len(Cs)):
  print(Cs[i])
  tic = timeit.default_timer()
  svm_model = svm.SVC(kernel='rbf', C=Cs[i]).fit(mnist_X_train, mnist_y_train)
  toc = timeit.default_timer()
  times[i] = toc - tic

  svm_pred = svm_model.predict(mnist_X_test)

  acc[i] = metrics.accuracy_score(mnist_y_test, svm_pred)
  f1[i] = metrics.f1_score(mnist_y_test, svm_pred, average='macro')

print('\nResultados')
print(Cs)
print(times)
print(acc)
print(f1)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Número de árboles en el ensemble
n = [10, 50, 100, 500, 1000]

# Profundidad máxima de cada árbol
max_depth_trees = [None, 3, 5]

times = [None] * len(n)
acc = [None] * len(n)
f1 = [None] * len(n)


for i in range(len(n)):
  print(n[i])
  times[i] = [-1] * len(max_depth_trees)
  acc[i] = [-1] * len(max_depth_trees)
  f1[i] = [-1] * len(max_depth_trees)
  for j in range(len(max_depth_trees)):
    print('  ' + str(max_depth_trees[j]))
    tic = timeit.default_timer()
    rf = RandomForestClassifier(n_estimators=n[i], max_depth=max_depth_trees[j]).fit(mnist_X_train, mnist_y_train)
    toc = timeit.default_timer()
    times[i][j] = toc - tic

    rf_pred = rf.predict(mnist_X_test)

    acc[i][j] = metrics.accuracy_score(mnist_y_test, rf_pred)
    f1[i][j] = metrics.f1_score(mnist_y_test, rf_pred, average='macro')

print('\nResultados')
print(times)
print(acc)
print(f1)

In [ ]:
from sklearn.neural_network import MLPClassifier
import math

n_attr = 784

# Distintas configuraciones de capas ocultas
# Cada tupla indica el número de neuronas en cada una de las capas ocultas
hidden = [(10,), (50,), (50, 10,), (100,), (100, 50,), (round(math.sqrt(n_attr)),), (round(math.sqrt(n_attr)), round(math.log2(n_attr)))]
times = [None] * len(hidden)
acc = [None] * len(hidden)
f1 = [None] * len(hidden)


for i in range(len(hidden)):
  print(hidden[i])
  tic = timeit.default_timer()
  mlp = MLPClassifier(hidden_layer_sizes=hidden[i], random_state=0).fit(mnist_X_train, mnist_y_train)
  toc = timeit.default_timer()

  times[i] = toc - tic

  mlp_pred = mlp.predict(mnist_X_test)

  acc[i] = metrics.accuracy_score(mnist_y_test, mlp_pred)
  f1[i] = metrics.f1_score(mnist_y_test, mlp_pred, average='macro')

print('\nResultados')
print(hidden)
print(times)
print(acc)
print(f1)


In [ ]:
import timeit

from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.naive_bayes import GaussianNB

# Entrenar clasificadores clásicos con conjunto de datos de mnist
tic = timeit.default_timer()
lr = LogisticRegression(random_state=0).fit(mnist_X_train, mnist_y_train)
toc = timeit.default_timer()
print('LR train time: ' + str(toc - tic))

lr_pred = lr.predict(mnist_X_test)

print('LR acc: ' + str(metrics.accuracy_score(mnist_y_test, lr_pred)))
print('LR F1: ' + str(metrics.f1_score(mnist_y_test, lr_pred, average='macro')))
print('---')

tic = timeit.default_timer()
tree = tree.DecisionTreeClassifier().fit(mnist_X_train, mnist_y_train)
toc = timeit.default_timer()
print('Tree train time: ' + str(toc - tic))

tree_pred = tree.predict(mnist_X_test)

print('Tree acc: ' + str(metrics.accuracy_score(mnist_y_test, tree_pred)))
print('Tree F1: ' + str(metrics.f1_score(mnist_y_test, tree_pred, average='macro')))
print('---')

tic = timeit.default_timer()
gnb = GaussianNB().fit(mnist_X_train, mnist_y_train)
toc = timeit.default_timer()
print('GNB train time: ' + str(toc - tic))

gnb_pred = gnb.predict(mnist_X_test)

print('GNB acc: ' + str(metrics.accuracy_score(mnist_y_test, gnb_pred)))
print('GNB F1: ' + str(metrics.f1_score(mnist_y_test, gnb_pred, average='macro')))
print('---')

In [ ]:
# Tener en cuenta que con ciertos conjuntos de datos puede ser MUY costoso obtener las predicciones en test
tic = timeit.default_timer()
knn = KNeighborsClassifier(n_neighbors=3).fit(mnist_X_train, mnist_y_train)
toc = timeit.default_timer()
print('kNN train time: ' + str(toc - tic))

tic = timeit.default_timer()
knn_pred = knn.predict(mnist_X_test)
toc = timeit.default_timer()
print('kNN testing time: ' + str(toc - tic))

print('kNN acc: ' + str(metrics.accuracy_score(mnist_y_test, knn_pred)))
print('kNN F1: ' + str(metrics.f1_score(mnist_y_test, knn_pred, average='macro')))
print('---')

Analice y compare los resultados obtenidos con MNIST y responda a las mismas preguntas propuestas anteriormente. ¿Los resultados son similares? ¿Ha respondido lo mismo a todas las preguntas?

Por último, copie y modifique el código necesario para utilizar el resto de conjuntos de datos propuestos al inicio del notebook.

De nuevo, vuelva a responder a las mismas preguntas y analice si está respondiendo lo mismo o no en base al conjunto de datos utilizado.

¿Son siempre mejor los métodos avanzados a los clásicos? ¿Son mejores los clásicos? ¿Depende del conjunto de datos?